In [3]:
#import danych i zastąpienie brakujących wartości modą
from ucimlrepo import fetch_ucirepo

heart_disease = fetch_ucirepo(id=45)

X = heart_disease.data.features
Y = heart_disease.data.targets

# print(heart_disease.variables)

X.fillna({'ca': X['ca'].median(), 'thal': X['thal'].mode()[0]}, inplace=True)

display(X)

c:\users\krzys\desktop\pythonproject\againterpreter\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0


In [4]:
#model klasyfikacji z regresją logistyczną

import numpy as np
from sklearn.model_selection import train_test_split

for index, row in Y.iterrows():
    if row[0] != 0:
        row[0] = 1 #jeżeli nie ma klasy 0 to ma klasę 1

y = Y.to_numpy()
X = (X-X.min())/(X.max()-X.min())
x = X.to_numpy()

print(x)
print(x.shape)
print(y.shape)

[[0.70833333 1.         0.         ... 1.         0.         0.75      ]
 [0.79166667 1.         1.         ... 0.5        1.         0.        ]
 [0.79166667 1.         1.         ... 0.5        0.66666667 1.        ]
 ...
 [0.58333333 1.         1.         ... 0.5        0.33333333 1.        ]
 [0.58333333 0.         0.33333333 ... 0.5        0.33333333 0.        ]
 [0.1875     1.         0.66666667 ... 0.         0.         0.        ]]
(303, 13)
(303, 1)


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2)

number_of_features = 13
#losowa inicjalizacja wag

weights = np.random.uniform(0.0, 100.0, size=number_of_features)
bias= np.random.rand(1)

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z/200))
    # return 1 / (1 + np.exp(-(z-200)))

def neuron_calculation(X, weights, bias):
    sig_arg = np.dot(X, weights) + bias
    return sigmoid(sig_arg)


In [7]:
predicted_probabilities = neuron_calculation(x_train, weights, bias)

print(predicted_probabilities)

[0.8034703  0.75774583 0.79395349 0.75838567 0.77948275 0.77525362
 0.81083218 0.70302536 0.76088332 0.80245412 0.69849167 0.70818971
 0.81202898 0.85518255 0.80472808 0.72004492 0.65956251 0.70590109
 0.6935855  0.75999214 0.60975632 0.80420863 0.78350356 0.85698294
 0.747237   0.633886   0.60920879 0.77403382 0.78883902 0.83157676
 0.78746444 0.61666697 0.70772551 0.7228303  0.66401601 0.76226734
 0.80606239 0.67852551 0.70264875 0.66084189 0.79416869 0.69935089
 0.67704422 0.84925633 0.79228736 0.78436363 0.78996299 0.75489577
 0.74616386 0.75746696 0.63127119 0.77201607 0.80606594 0.84704067
 0.74404026 0.79400223 0.78637965 0.76624212 0.673589   0.78498651
 0.85362887 0.86677345 0.69061662 0.87172641 0.69738226 0.68922768
 0.84196411 0.66206748 0.77192631 0.78941178 0.62261005 0.79573041
 0.62856454 0.88273357 0.63242911 0.70769064 0.71878961 0.68773216
 0.67535622 0.83371064 0.59724566 0.63456928 0.80543532 0.83425308
 0.76767768 0.78014774 0.82795483 0.80639345 0.71867589 0.6770

In [8]:
# ta funkcja kosztu ma
def loss_fun(y, y_pred):
    epsilon = 1e-15  # zapobieganie log(0)
    loss = - (y * np.log(y_pred + epsilon) + (1 - y) * np.log(1 - y_pred + epsilon))
    # loss = - (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred ))

    return loss


def update_weights(X, y, y_pred, weights, bias, learning_rate):
    arX = np.squeeze(np.asarray(X))
    ary = np.squeeze(np.asarray((y_pred - y)))
    gradient_weights = np.dot(arX, ary)
    gradient_bias = np.sum(y_pred - y)
    #aktualizacja wag
    weights -= learning_rate * gradient_weights
    bias -= learning_rate * gradient_bias

    return weights, bias

In [9]:
# print(np.mean(loss_fun(y_train[1], sigmoid(np.dot(x_train[1], weights) + bias))))
# print(loss_fun(y_train[1], sigmoid(np.dot(x_train[1], weights) + bias)))
print(type(x_train))
print(weights)

<class 'numpy.ndarray'>
[26.99201174 43.56794124  1.38222621 58.12122119 92.64274237 85.79376628
 96.36466482 80.04343412 26.60320607  3.50027929  8.69026012 28.02512599
  0.860178  ]


In [10]:
num_of_features = x_train.shape[1]
# hiperparametry
learning_rate = 0.3
epochs = 2000
prev_loss = 1
avg_loss = 1

for epoch in range(epochs):
    loss_arr = []
    for i in range(len(x_train)):
        X = x_train[i]
        y = y_train[i]
        y_pred = sigmoid(np.dot(X, weights) + bias)
        loss = loss_fun(y, y_pred)
        loss_arr.append(loss)
        weights, bias = update_weights(X, y, y_pred, weights, bias, learning_rate)

        binary_prediction = 1 if y_pred >= 0.5 else 0
    if epoch % 100 == 0:
        prev_loss = avg_loss
        avg_loss = sum(loss_arr) / len(loss_arr)
        print(f"Epoch {epoch}: Average loss = {avg_loss}")
        #if (1-(avg_loss/prev_loss))*100 < 2: break #zbyt mała zmiana f.kosztu procentowo

print("Trained Weights:", weights)
print("Trained Bias:", bias)


Epoch 0: Average loss = [0.80199707]
Epoch 100: Average loss = [0.4176474]
Epoch 200: Average loss = [0.38366244]
Epoch 300: Average loss = [0.37009379]
Epoch 400: Average loss = [0.36273616]
Epoch 500: Average loss = [0.35812195]
Epoch 600: Average loss = [0.35495125]
Epoch 700: Average loss = [0.35263111]
Epoch 800: Average loss = [0.3508561]
Epoch 900: Average loss = [0.34945406]
Epoch 1000: Average loss = [0.34832052]
Epoch 1100: Average loss = [0.34738807]
Epoch 1200: Average loss = [0.3466109]
Epoch 1300: Average loss = [0.34595645]
Epoch 1400: Average loss = [0.34540077]
Epoch 1500: Average loss = [0.3449257]
Epoch 1600: Average loss = [0.34451717]
Epoch 1700: Average loss = [0.34416405]
Epoch 1800: Average loss = [0.34385741]
Epoch 1900: Average loss = [0.34359004]
Trained Weights: [-147.50888429  195.15165623  319.06185165  414.63414838  153.2702821
 -177.03797546  135.73626987 -462.85866607  165.28643887  485.96693823
  258.19407125  775.80666933  295.97479826]
Trained Bias: 

In [11]:
def predict(x_data):
    y_preds = []
    for i in range(len(x_data)):
        X = x_data[i]
        y_pred = sigmoid(np.dot(X, weights) + bias)
        print(y_pred)
        binary_prediction = 1 if y_pred >= 0.5 else 0
        y_preds.append(binary_prediction)
    return np.array(y_preds)

In [12]:
#Ocena modelu
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred = predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Confusion: {confusion}')
print(f'Report: {report}')

[0.26137979]
[0.3235555]
[0.72030682]
[0.08811941]
[0.04674313]
[0.04761401]
[0.66470726]
[0.02739183]
[0.04486012]
[0.99632406]
[0.94853401]
[0.98178307]
[0.05366666]
[0.21845639]
[0.88876231]
[0.29028641]
[0.03715324]
[0.05396505]
[0.01617608]
[0.97888807]
[0.93006019]
[0.9775171]
[0.12713025]
[0.74286826]
[0.14373727]
[0.21336072]
[0.00983207]
[0.12097168]
[0.17122704]
[0.79362191]
[0.18026331]
[0.70255519]
[0.9584979]
[0.04924141]
[0.17963655]
[0.4889384]
[0.98211885]
[0.0409725]
[0.0289731]
[0.05353171]
[0.8752077]
[0.86750771]
[0.97041881]
[0.04877747]
[0.45810077]
[0.10730786]
[0.7780691]
[0.04029871]
[0.94918904]
[0.11247264]
[0.17917382]
[0.9411317]
[0.99403535]
[0.9520936]
[0.99657562]
[0.91886855]
[0.99089492]
[0.02630096]
[0.95349865]
[0.9859725]
[0.78916464]
Accuracy: 0.8524590163934426
Confusion: [[28  4]
 [ 5 24]]
Report:               precision    recall  f1-score   support

           0       0.85      0.88      0.86        32
           1       0.86      0.83      0.8

In [19]:
#Ocena modelu
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
y_pred = predict(x_train)

accuracy = accuracy_score(y_train, y_pred)
confusion = confusion_matrix(y_train, y_pred)
report = classification_report(y_train, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Confusion: {confusion}')
print(f'Report: {report}')

Accuracy: 0.8512396694214877
Confusion: [[111  14]
 [ 22  95]]
Report:               precision    recall  f1-score   support

           0       0.83      0.89      0.86       125
           1       0.87      0.81      0.84       117

    accuracy                           0.85       242
   macro avg       0.85      0.85      0.85       242
weighted avg       0.85      0.85      0.85       242

